In [25]:
import pandas as pd
import json
import os
from statsbombpy import sb

In [26]:
sb.competitions()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882
...,...,...,...,...,...,...,...,...,...,...,...,...
69,55,43,Europe,UEFA Euro,male,False,True,2020,2024-04-16T12:44:40.558402,2024-04-16T12:47:18.505110,2024-04-16T12:47:18.505110,2024-04-16T12:44:40.558402
70,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2024-02-12T14:45:05.702250,2021-06-13T16:17:31.694,None,2024-02-12T14:45:05.702250
71,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2024-02-13T13:27:17.178263,2024-02-13T13:30:52.820588,2024-02-13T13:30:52.820588,2024-02-13T13:27:17.178263
72,72,107,International,Women's World Cup,female,False,True,2023,2024-07-14T16:59:48.469596,2024-07-14T17:01:38.887279,2024-07-14T17:01:38.887279,2024-07-14T16:59:48.469596


In [28]:
euros_2024_competition_id = 55
euros_2024_season_id = 282
matches = sb.matches(competition_id=euros_2024_competition_id, season_id=euros_2024_season_id)

In [32]:
spain_matches = matches[(matches['home_team'] == 'Spain') | (matches['away_team'] == 'Spain')]
for _, match in spain_matches.iterrows():
    match_id = match['match_id']
    lineups = sb.lineups(match_id=match_id)
    
    print(f"Match ID: {match_id}")
    print(f"{match['home_team']} vs {match['away_team']}")
    print("Home Team Lineup:")
    print(lineups[match['home_team']])
    print("\nAway Team Lineup:")
    print(lineups[match['away_team']])
    print("\n" + "="*50 + "\n")

Match ID: 3943043
Spain vs England
Home Team Lineup:
    player_id                      player_name   player_nickname  \
0        3042               Mikel Merino Zazón      Mikel Merino   
1        3265         José Luis Sanmartín Mato            Joselu   
2        3477       Álvaro Borja Morata Martín     Álvaro Morata   
3        4127                David Raya Martin        David Raya   
4        4353                  Aymeric Laporte              None   
5        5202  José Ignacio Fernández Iglesias             Nacho   
6        5721            Daniel Carvajal Ramos   Daniel Carvajal   
7        6655                 Fabián Ruiz Peña       Fabián Ruiz   
8        6685           Mikel Oyarzabal Ugarte   Mikel Oyarzabal   
9        6748             Ferrán Torres García     Ferrán Torres   
10       6765       Rodrigo Hernández Cascante             Rodri   
11       6821             Jesús Navas González       Jesús Navas   
12      10336        Alejandro Grimaldo García     Álex Grimald

In [42]:
lamine_yamal_id = 316046
matches_with_lamine_yamal = []
player_match = sb.player_match_stats(lamine_yamal_id)
player_season = sb.player_season_stats(competition_id=9, season_id=42)
team_season = sb.team_season_stats(competition_id=9, season_id=42)

player_match

Exception: There is currently no open data for aggregated stats, please provide credentials